In [1]:
%store

Stored variables and their in-db values:
__importClassification             -> IPython.macro.Macro(u"# Handle table-like data and
__importRegression                 -> IPython.macro.Macro(u"# Handle table-like data and


In [2]:
%store -r __importRegression

In [3]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
endMonth = 29
fnameTest = '../input/validation/test_' + str(endMonth) + '.csv'
fnameTrain = '../input/validation/train_' + str(endMonth) + '.csv'
train = pd.read_csv(fnameTrain)
test = pd.read_csv(fnameTest)
items = pd.read_csv("../input/items.csv")
categories = pd.read_csv("../input/item_categories.csv")

print (train.shape, test.shape, items.shape, categories.shape)

((2664552, 6), (31909, 3), (22170, 3), (84, 2))


In [5]:
labels = test.pop('item_cnt_day')

In [6]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
test.head()

,shop_id,item_id
0,2,486
1,2,792
2,2,949
3,2,950
4,2,1090


In [8]:
train.pop('date')
train.pop('date_block_num')
train.pop('item_price');

In [9]:
train.head()

,shop_id,item_id,item_cnt_day
0,59,22154,1.0
1,25,2552,1.0
2,25,2552,-1.0
3,25,2554,1.0
4,25,2555,1.0


In [10]:
train = train.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day':sum})

In [11]:
print train.shape
train.head()

(396546, 3)


,shop_id,item_id,item_cnt_day
0,0,30,31.0
1,0,31,11.0
2,0,32,16.0
3,0,33,6.0
4,0,35,15.0


In [12]:
train.item_cnt_day.describe()

count    396546.000000
mean          8.349415
std          59.686950
min         -12.000000
25%           1.000000
50%           3.000000
75%           7.000000
max       17678.000000
Name: item_cnt_day, dtype: float64

Take average sales in 33 months and remove duplicates

In [13]:
train.item_cnt_day = train.item_cnt_day / (endMonth-1)

In [14]:
train.head()

,shop_id,item_id,item_cnt_day
0,0,30,1.107143
1,0,31,0.392857
2,0,32,0.571429
3,0,33,0.214286
4,0,35,0.535714


In [15]:
train.drop_duplicates(inplace=True)
train.shape

(396546, 3)

In [16]:
test.head()

,shop_id,item_id
0,2,486
1,2,792
2,2,949
3,2,950
4,2,1090


In [17]:
train_labels = train.pop('item_cnt_day')

In [18]:
#test_id = test.pop('ID')

# LightGBM tunning

LBC = LGBMRegressor()
lb_param_grid = {
              'n_estimators' : [10, 50, 100],
              'learning_rate': [0.1, 0.01],
              'max_depth': [4, 8]
              }

lGBC = GridSearchCV(LBC,param_grid = lb_param_grid, cv=5,
                     scoring="neg_mean_squared_error", n_jobs= -1, verbose = 1)

lGBC.fit(train, train_labels)

lgbm = lGBC.best_estimator_

# Best score
lGBC.best_score_

In [19]:
lgbm = LGBMRegressor()
lgbm.fit(train, train_labels)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
       objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [20]:
predictions = lgbm.predict(test)
predictions.shape

(31909,)

In [21]:
labels.shape

(31909,)

Clipping predictions in the [0, 20] range

In [22]:
for i in range(len(predictions)):
    if predictions[i] > 20:
        predictions[i] = 20
    if predictions[i] < 0:
        predictions[i] = 20

for i in range(len(labels)):
    if labels[i] > 20:
        labels[i] = 20
    if labels[i] < 0:
        labels[i] = 20

Compute RMSE

In [23]:
np.sqrt(mean_squared_error(labels, predictions))

6.064427852580307

## Submission

test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': predictions})
print test_submit.shape
test_submit.to_csv('lgbm_all_train.csv', index=False)
test_submit.head()

test_submit.item_cnt_month.describe()